<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Interacting with Tables in Storage

## Azure Actions Covered

* Creating tables in Azure storage
* Interacting with tables and entities in Azure storage
* Querying entities in Azure storage
* Deleting entities and tables in Azure storage

In this lecture, we're going to take a look at how to interact with tables in storage containers via the Python SDK.

To begin, we import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). 

In [1]:
from datetime import datetime

from azure.identity import AzureCliCredential
# New imports for tables
from azure.data.tables import TableServiceClient, TableClient

from settings import AZURE_SUBSCRIPTION_ID

In [2]:
credential = AzureCliCredential()

To interact with tables, we'll need to instantiate a `TableServiceClient`. The `endpoint` parameter will be of the form `https://<storage-account>.table.core.windows.net`.

In [4]:
table_service_client = TableServiceClient(
    endpoint='https://benbstorage1234.table.core.windows.net',
    credential=credential
)

We can use that service client to create a new table. This will return a `TableClient` object for interacting with the table.

In [5]:
table_client = table_service_client.create_table(table_name='bensTable')

Let's look at some of the properties of our table client, like the URL.

In [6]:
table_client.url

'https://benbstorage1234.table.core.windows.net'

We can insert entities into our new table via the table client and the `create_entity()` method. Make sure to include a `PartitionKey` and a `RowKey` in addition to any other information. We'll make up some healthcare information as a test.

In [9]:
entity = table_client.create_entity(
    entity={
        'PartitionKey': 'Doe, John',
        'RowKey': '00001',
        'AdmissionDate': datetime(2023, 5, 10),
        'DischargeDate': datetime(2023, 5, 15),
        'Age': 78,
        'Diagnosis': 'Flu'
    }
)

Now let's list all the entities in our table.

In [10]:
for entity in table_client.list_entities():
    print(entity)

{'PartitionKey': 'Doe, John', 'RowKey': '00001', 'AdmissionDate': TablesEntityDatetime(2023, 5, 10, 6, 0, tzinfo=datetime.timezone.utc), 'DischargeDate': TablesEntityDatetime(2023, 5, 15, 6, 0, tzinfo=datetime.timezone.utc), 'Age': 78, 'Diagnosis': 'Flu'}


Because tables can have many entities, you may want to filter it down to a distinct group of data. We can do this using the `query_entities()` method on our table client. This will take:
* `query_filter` - Query to use to filter data from the table
* `parameters` - Parameters for the query filter

The parameters can easily be stored in a dictionary so that you can reference the key values in the query filter string. The query filter must also refer to keys of entities in the table, and parameters from the dictionary can be referenced with the `@` symbol.

In [13]:
parameters = {
    'admission_date': datetime(2023, 5, 1)
}
query_filter = "AdmissionDate ge @admission_date"
result = table_client.query_entities(
    query_filter=query_filter,
    parameters=parameters
)
for item in result:
    print(item)

{'PartitionKey': 'Doe, John', 'RowKey': '00001', 'AdmissionDate': TablesEntityDatetime(2023, 5, 10, 6, 0, tzinfo=datetime.timezone.utc), 'DischargeDate': TablesEntityDatetime(2023, 5, 15, 6, 0, tzinfo=datetime.timezone.utc), 'Age': 78, 'Diagnosis': 'Flu'}


Let's try filtering on the `PartitionKey` as well.

In [14]:
parameters = {
    'pk': 'Doe, John'
}
query_filter = "PartitionKey eq @pk"
result = table_client.query_entities(
    query_filter=query_filter,
    parameters=parameters
)
for item in result:
    print(item)

{'PartitionKey': 'Doe, John', 'RowKey': '00001', 'AdmissionDate': TablesEntityDatetime(2023, 5, 10, 6, 0, tzinfo=datetime.timezone.utc), 'DischargeDate': TablesEntityDatetime(2023, 5, 15, 6, 0, tzinfo=datetime.timezone.utc), 'Age': 78, 'Diagnosis': 'Flu'}


For more information on how to write filters for data from tables, see the [Azure documentation on GitHub.](https://github.com/Azure/azure-sdk-for-python/tree/azure-data-tables_12.4.2/sdk/tables/azure-data-tables/samples#writing-filters).

We can delete entities in the table as well by specifying the row and partition keys.

In [15]:
table_client.delete_entity(
    partition_key='Doe, John',
    row_key='00001'
)

Now there's nothing left in our table.

In [16]:
for entity in table_client.list_entities():
    print(entity)

Just like we can list entities, we can list tables in our storage account via the table service client.

In [18]:
for table in table_service_client.list_tables():
    print(table.name)

bensTable


Just like we can query the entities in the table, we can query the tables in our storage account using the same combination of query filters and parameters.

In [19]:
parameters = {
    'table_name': 'bensTable'
}
query_filter = 'TableName eq @table_name'
result = table_service_client.query_tables(
    query_filter=query_filter,
    parameters=parameters
)
for table in result:
    print(table.name)

bensTable


Finally, let's clean up our storage account by deleting our table.

In [20]:
table_service_client.delete_table(table_name='bensTable')